In [5]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
#Reading csv file and selecting 500 samoles and creating a data csv file and checking its top files

df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
281,"It's too bad these guys, the so-called judges,...",negative
408,Ladies and gentlemen: the show begins with thi...,positive
826,I guess this is the first movie that made me a...,positive
384,Considering 'A Star is Born' had been made twi...,positive
0,Film version of Sandra Bernhard's one-woman of...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    # Lemmatization is a text normalization technique used to reduce inflected words to their base or dictionary form, known as the lemma
    """Lemmatize the text.""" 
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    #These words, such as "a," "the," "is," "and," etc., are typically removed during text preprocessing to reduce noise, improve model performance, and focus on more meaningful terms.
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
281,bad guy so called judge jerk even nominally se...,negative
408,lady gentleman show begin documentary film str...,positive
826,guess first movie made aware term cult flick t...,positive
384,considering a star born made twice already tim...,positive
0,film version sandra bernhard s one woman off b...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    267
positive    233
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
281,bad guy so called judge jerk even nominally se...,0
408,lady gentleman show begin documentary film str...,1
826,guess first movie made aware term cult flick t...,1
384,considering a star born made twice already tim...,1
0,film version sandra bernhard s one woman off b...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/adeelajmal2468/MLOPS-Capstone-Project.mlflow')
dagshub.init(repo_owner='adeelajmal2468', repo_name='MLOPS-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-11-12 16:04:23,996 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/adeelajmal2468/MLOPS-Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "adeelajmal2468/MLOPS-Capstone-Project"

2025-11-12 16:04:23,996 - INFO - Initialized MLflow to track repo "adeelajmal2468/MLOPS-Capstone-Project"


Repository adeelajmal2468/MLOPS-Capstone-Project initialized!

2025-11-12 16:04:23,996 - INFO - Repository adeelajmal2468/MLOPS-Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/8f95e25e0a2b499eafab8f3f0c789855', creation_time=1762944884725, experiment_id='0', last_update_time=1762944884725, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-12 16:04:27,517 - INFO - Starting MLflow run...
2025-11-12 16:04:27,962 - INFO - Logging preprocessing parameters...
2025-11-12 16:04:28,913 - INFO - Initializing Logistic Regression model...
2025-11-12 16:04:28,913 - INFO - Fitting the model...
2025-11-12 16:04:28,927 - INFO - Model training complete.
2025-11-12 16:04:28,927 - INFO - Logging model parameters...
2025-11-12 16:04:29,358 - INFO - Making predictions...
2025-11-12 16:04:29,358 - INFO - Calculating evaluation metrics...
2025-11-12 16:04:29,368 - INFO - Logging evaluation metrics...
2025-11-12 16:04:30,697 - INFO - Saving and logging the model...
2025/11/12 16:04:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-11-12 16:04:36,804 - INFO - Model training and logging completed in 8.84 seconds.
2025-11-12 16:04:36,805 - INFO - Accuracy: 0.624
2025-11-12 16:04:36,805 - INFO - Precision: 0

🏃 View run bright-doe-582 at: https://dagshub.com/adeelajmal2468/MLOPS-Capstone-Project.mlflow/#/experiments/0/runs/10d9ca6cb23c4bfeb8a5f4016511568c
🧪 View experiment at: https://dagshub.com/adeelajmal2468/MLOPS-Capstone-Project.mlflow/#/experiments/0
